In [2]:
!pip install openpyxl
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from joblib import dump, load

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


C:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Python311\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
svm = load('SVM_big_dataset.joblib')

In [19]:
data = pd.read_csv('walkingDead.csv')
data.dropna(subset=['sentences'], inplace=True)

In [5]:
import spacy
import string
nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words
punctuations = string.punctuation

In [6]:
def spacy_tokenizer(sentence):
    if pd.isna(sentence):
        return
    else:
        doc = nlp(sentence)
        mytokens = [ word.lemma_.lower().strip() for word in doc ] 
        mytokens = [ word for word in mytokens if word not in punctuations ] 
        sentence = " ".join(mytokens) # return preprocessed list of tokens return sentence
        return sentence

In [20]:
data['tokenize'] = data['sentences'].apply(spacy_tokenizer)
print(len(data['tokenize']))

977


In [21]:
data['embeddings'] = data['tokenize'].apply(model.encode)

In [24]:
import openpyxl
from openpyxl.styles import PatternFill
X = data['embeddings'].to_list()
predicted = svm.predict(X)
data['predicted'] = predicted

In [25]:
data.to_excel('walkingDeadPredictions.xlsx', index=False)

In [26]:
wb = openpyxl.load_workbook('walkingDeadPredictions.xlsx')
ws = wb.active

predicted_column_index = data.columns.get_loc('predicted') + 1
# Set red color fill for cells where predicted value is 1
for i, value in enumerate(predicted, start=2):
    cell = ws.cell(row=i, column=predicted_column_index)
    if value == 1:
        cell.fill = PatternFill(start_color='FF0000', end_color='FF0000', fill_type='solid')

wb.save('walkingDeadPredictions.xlsx')